In [2]:
import cv2
import json
import numpy as np
import os.path as osp
from time import time
from pathlib import Path
from tqdm import tqdm

from core import ObjectDetector, CornerPredictor

In [3]:
dt_vehicle = ObjectDetector('yolov4-default')
dt_plate   = ObjectDetector('yolov4-plate_type')
dt_number  = ObjectDetector('yolov4-plate_number')
dt_corner  = CornerPredictor('corner_prediction')

<yolov4-default> model loaded.
<yolov4-plate_type> model loaded.
<yolov4-plate_number> model loaded.
<corner_prediction> model loaded.


In [7]:
def start(img):
    obj2, number = '', ''
    t1, t2, t3, t4 = 0, 0, 0, 0

    s_time = time()
    _ = dt_vehicle.loadImage(img)
    bbox1, _ = dt_vehicle.runDetection(mode='vehicle')
    t1 = time() - s_time # vehicle detection time

    if len(bbox1) != 0:
        s_time = time()
        img_vehicle = dt_plate.loadImage(img, True, bbox1)
        bbox2, obj2 = dt_plate.runDetection(mode='plate')
        t2 = time() - s_time # LP detection time

        if len(bbox2) != 0:
            s_time = time()
            _ = dt_corner.loadImage(img_vehicle, True, bbox2)
            img_plate_warped, _ = dt_corner.runPrediction()
            t3 = time() - s_time # LP corner prediction time

            s_time = time()
            _ = dt_number.loadImage(img_plate_warped)
            bbox3, obj3 = dt_number.runDetection(mode='number', multi_res=True)

            if len(bbox3) != 0:
                order = bbox3[:, 0, 0].argsort()
                for o in order:
                    number += obj3[o]
                t4 = time() - s_time # LP recognition time

    return obj2, number, t1, t2, t3, t4

def eval(test_path):
    n_test = len(test_path)
    total_time = np.array([0., 0., 0., 0.])
    num_result = 0
    initial = True

    for i in tqdm(range(n_test), unit=' images', ncols=100):
        p = test_path[i]
        img = cv2.cvtColor(cv2.imread(p), cv2.COLOR_BGR2RGB)
        _, d_number, t1, t2, t3, t4 = start(img)

        if d_number != '':
            num_result += 1
            if initial:
                _, _, t1, t2, t3, t4 = start(img)
                initial = False
            total_time[0] += t1
            total_time[1] += t2
            total_time[2] += t3
            total_time[3] += t4

    return total_time, num_result

In [5]:
dataset_dir1 = r'D:\User Profiles\Desktop\Personal Files\School\University Project\myPlateDetection\all_dataset\test_data'
vehicle_dataset1 = osp.join(dataset_dir1, 'vehicle_images')

test_path1 = [str(p) for p in Path(vehicle_dataset1).glob('**/*.jpg')]
print('total:', len(test_path1))

total: 2049


In [6]:
dataset_dir2 = r'D:\User Profiles\Desktop\Personal Files\School\University Project\myPlateDetection\all_dataset\train_valid_data'
vehicle_dataset2 = osp.join(dataset_dir2, 'vehicle_images')

path_file = r'D:\User Profiles\Desktop\Personal Files\School\University Project\myPlateDetection\training_data\plate_number_detection\file_path.json'
with open(path_file, 'r') as f:
    test_path2 = json.load(f)['valid']

test_path2 = [osp.join(vehicle_dataset2, osp.basename(p).split('-')[0]+'.jpg') for p in test_path2]
print('total:', len(test_path2))

total: 3496


In [8]:
total_time = np.array([0., 0., 0., 0.])
num_result = 0

t_time, n_result = eval(test_path1) # AOLP
total_time += t_time
num_result += n_result

t_time, n_result = eval(test_path2) # Ours
total_time += t_time
num_result += n_result

print('Results:', num_result)
print('Vehicle detection time: {:.3f}s'.format(total_time[0]/num_result))
print('LP detection time: {:.3f}s'.format(total_time[1]/num_result))
print('LP corner prediction time: {:.3f}s'.format(total_time[2]/num_result))
print('LP recognition time: {:.3f}s'.format(total_time[3]/num_result))
print('Total processing time: {:.3f}s'.format(sum(total_time)/num_result))

100%|██████████████████████████████████████████████████████| 3496/3496 [05:31<00:00, 10.55 images/s]

Results: 3313
Vehicle detection time: 0.040s
LP detection time: 0.036s
LP corner prediction time: 0.003s
LP recognition time: 0.034s
Total processing time: 0.113s
